<a href="https://colab.research.google.com/github/hamdansyaif/DeepLearning/blob/main/Hands-On-ML2/Chapter-03/Chapter_3_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🔧 Instalasi library yang dibutuhkan untuk Hands-on ML Chapter 3
!pip install --upgrade pip
!pip install numpy matplotlib scikit-learn pandas
!pip install numpy matplotlib scikit-learn
!pip install scipy

In [ ]:
!nvidia-smi

In [ ]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib
import sys

print("Python version:", sys.version)
print("Scikit-learn:", sklearn.__version__)
print("NumPy:", np.__version__)
print("Pandas:", pd.__version__)
print("Matplotlib:", matplotlib.__version__)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml

In [ ]:
np.random.seed(42)

In [ ]:
from sklearn.datasets import fetch_openml

# Ambil ulang data MNIST
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist["data"], mnist["target"]
y = y.astype(np.uint8)

# Bagi data menjadi training dan test (60k + 10k)
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]


In [ ]:
y = y.astype(np.uint8)

In [ ]:
print(f"Ukuran fitur (X): {X.shape}")
print(f"Ukuran label (y): {y.shape}")

In [ ]:
# --- AMBIL CONTOH GAMBAR PERTAMA ---
some_digit = X[0]
some_digit_image = some_digit.reshape(28, 28)  # Ubah kembali ke 2D untuk ditampilkan

# --- TAMPILKAN GAMBAR ---
plt.imshow(some_digit_image, cmap="binary")
plt.axis("off")
plt.title(f"Label: {y[0]}")
plt.show()


In [ ]:
# Target: True jika angka 5, False jika bukan
y_binary = (y == 5)

# Pisahkan train-test (data MNIST: 60.000 train pertama, 10.000 sisanya test)
X_train, X_test = X[:60000], X[60000:]
y_train_binary, y_test_binary = y_binary[:60000], y_binary[60000:]


In [ ]:
from sklearn.linear_model import SGDClassifier

# Inisialisasi model SGD
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_binary)  # training hanya untuk deteksi angka 5


In [ ]:
prediction = sgd_clf.predict([some_digit])
print("Apakah ini angka 5? :", prediction[0])


In [ ]:
from sklearn.model_selection import cross_val_score

# Evaluasi dengan 3-fold cross-validation
cv_scores = cross_val_score(sgd_clf, X_train, y_train_binary, cv=3, scoring="accuracy")
print("Cross-Validation Accuracy (3 fold):", cv_scores)
print("Mean Accuracy:", cv_scores.mean())


In [ ]:
from sklearn.base import BaseEstimator

class Never5Classifier(BaseEstimator):
    def fit(self, X, y=None):
        return self
    def predict(self, X):
        return np.zeros((len(X),), dtype=bool)  # Selalu prediksi "bukan 5"

never_5_clf = Never5Classifier()
dummy_scores = cross_val_score(never_5_clf, X_train, y_train_binary, cv=3, scoring="accuracy")

print("Dummy Classifier Accuracy (Never 5):", dummy_scores)
print("Mean Dummy Accuracy:", dummy_scores.mean())


In [ ]:
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_binary, cv=3)


In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_train_binary, y_train_pred)
print("Confusion Matrix:\n", cm)


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_train_binary, y_train_pred)
recall = recall_score(y_train_binary, y_train_pred)
f1 = f1_score(y_train_binary, y_train_pred)

print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")


In [ ]:
# Ambil skor decision function (bukan hasil klasifikasi binary)
y_scores = cross_val_predict(sgd_clf, X_train, y_train_binary, cv=3, method="decision_function")


In [ ]:
from sklearn.metrics import precision_recall_curve

precisions, recalls, thresholds = precision_recall_curve(y_train_binary, y_scores)


In [ ]:
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall")
    plt.xlabel("Threshold")
    plt.legend(loc="center right")
    plt.grid(True)
    plt.title("Precision & Recall vs Threshold")

plt.figure(figsize=(8, 5))
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.show()


In [ ]:
# Ambil threshold pertama yang mencapai minimal 90% precision
threshold_90_precision = thresholds[np.argmax(precisions >= 0.90)]
print("Threshold untuk precision ≥ 90%:", threshold_90_precision)


In [ ]:
y_train_pred_90 = (y_scores >= threshold_90_precision)

from sklearn.metrics import precision_score, recall_score

print("Precision (≥90% threshold):", precision_score(y_train_binary, y_train_pred_90))
print("Recall (≥90% threshold):", recall_score(y_train_binary, y_train_pred_90))


In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, roc_thresholds = roc_curve(y_train_binary, y_scores)

def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label if label else "ROC")
    plt.plot([0, 1], [0, 1], 'k--')  # garis acak
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate (Recall)")
    plt.title("ROC Curve")
    plt.grid(True)

plt.figure(figsize=(8, 5))
plot_roc_curve(fpr, tpr)
plt.show()


In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(y_train_binary, y_scores)
print("ROC AUC Score:", roc_auc)


In [ ]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train)


In [ ]:
# Prediksi untuk some_digit (yang tadi kita tampilkan)
sgd_pred = sgd_clf.predict([some_digit])
print("Prediksi angka:", sgd_pred[0])
print("Label asli:", y[0])


In [ ]:
from sklearn.model_selection import cross_val_score

sgd_scores = cross_val_score(sgd_clf, X_train, y_train, cv=3, scoring="accuracy")
print("Multiclass Accuracy:", sgd_scores)
print("Rata-rata Akurasi:", sgd_scores.mean())


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))

sgd_scores_scaled = cross_val_score(sgd_clf, X_train_scaled, y_train, cv=3, scoring="accuracy")
print("Akurasi setelah scaling:", sgd_scores_scaled)
print("Rata-rata akurasi:", sgd_scores_scaled.mean())


In [ ]:
from sklearn.svm import SVC

svc_clf = SVC(gamma="auto")
svc_clf.fit(X_train[:1000], y_train[:1000])  # subset kecil karena lambat

svc_pred = svc_clf.predict([some_digit])
print("Prediksi SVC:", svc_pred[0])


In [ ]:
from sklearn.model_selection import cross_val_predict

# Gunakan model SGDClassifier yang sudah dilatih
y_train_pred = cross_val_predict(sgd_clf, X_train_scaled, y_train, cv=3)


In [ ]:
from sklearn.metrics import confusion_matrix

conf_mx = confusion_matrix(y_train, y_train_pred)
print("Confusion Matrix:\n", conf_mx)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(9,9))
plt.matshow(conf_mx, cmap=plt.cm.gray, fignum=1)
plt.title("Confusion Matrix (gray scale)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.colorbar()
plt.show()


In [ ]:
# Hitung rasio per baris (jumlah total label sesungguhnya)
row_sums = conf_mx.sum(axis=1, keepdims=True)
norm_conf_mx = conf_mx / row_sums

# Set diagonal (benar prediksi) ke 0 agar fokus ke kesalahan
np.fill_diagonal(norm_conf_mx, 0)

plt.figure(figsize=(9,9))
plt.matshow(norm_conf_mx, cmap=plt.cm.gray)
plt.title("Normalized Confusion Matrix (Error Only)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.colorbar()
plt.show()


In [ ]:
def plot_digits(instances, images_per_row=10):
    size = 28
    images_per_row = min(images_per_row, len(instances))
    n_rows = (len(instances) - 1) // images_per_row + 1
    padded = np.concatenate([instances, np.zeros((n_rows * images_per_row - len(instances), 784))])
    images = padded.reshape((n_rows, images_per_row, size, size))
    big_image = images.transpose(0, 2, 1, 3).reshape(n_rows * size, images_per_row * size)
    plt.imshow(big_image, cmap=plt.cm.binary)
    plt.axis("off")

cl_a, cl_b = 3, 5
X_aa = X_train_scaled[(y_train == cl_a) & (y_train_pred == cl_a)]
X_ab = X_train_scaled[(y_train == cl_a) & (y_train_pred == cl_b)]
X_ba = X_train_scaled[(y_train == cl_b) & (y_train_pred == cl_a)]
X_bb = X_train_scaled[(y_train == cl_b) & (y_train_pred == cl_b)]

plt.figure(figsize=(8,8))
plt.subplot(221); plot_digits(X_aa[:25], images_per_row=5); plt.title(f"{cl_a} correctly classified")
plt.subplot(222); plot_digits(X_ab[:25], images_per_row=5); plt.title(f"{cl_a} misclassified as {cl_b}")
plt.subplot(223); plot_digits(X_ba[:25], images_per_row=5); plt.title(f"{cl_b} misclassified as {cl_a}")
plt.subplot(224); plot_digits(X_bb[:25], images_per_row=5); plt.title(f"{cl_b} correctly classified")
plt.tight_layout()
plt.show()


In [ ]:
# Label 1: True jika angka ≥ 7
y_train_large = (y_train >= 7)

# Label 2: True jika angka ganjil
y_train_odd = (y_train % 2 == 1)

# Gabungkan jadi multilabel (dua kolom)
y_multilabel = np.c_[y_train_large, y_train_odd]
print("Contoh multilabel untuk angka", y_train[0], ":", y_multilabel[0])


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_multilabel)


In [ ]:
some_digit_pred = knn_clf.predict([X_train[0]])
print("Label asli:", y_multilabel[0])
print("Prediksi   :", some_digit_pred[0])


In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score

y_train_pred_multi = cross_val_predict(knn_clf, X_train, y_multilabel, cv=3)
f1 = f1_score(y_multilabel, y_train_pred_multi, average="macro")
print("F1 Score (Multilabel - Macro):", f1)


In [ ]:
# Tambahkan noise acak 0–99 ke semua piksel
noise = np.random.randint(0, 100, (len(X_train), 784))
X_train_noisy = X_train + noise

noise_test = np.random.randint(0, 100, (len(X_test), 784))
X_test_noisy = X_test + noise_test

# Label target = gambar asli (tanpa noise)
y_train_denoise = X_train
y_test_denoise = X_test


In [ ]:
knn_clf.fit(X_train_noisy, y_train_denoise)


In [ ]:
# Uji dengan satu contoh noisy
some_index = 0
noisy_digit = X_test_noisy[some_index]
clean_digit = knn_clf.predict([noisy_digit])

# Tampilkan hasil
def plot_digit(data):
    plt.imshow(data.reshape(28, 28), cmap="binary")
    plt.axis("off")

plt.subplot(1, 2, 1)
plot_digit(noisy_digit)
plt.title("Noisy")

plt.subplot(1, 2, 2)
plot_digit(clean_digit)
plt.title("Denoised")

plt.show()


# **Exercise**

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

param_grid = [{'weights': ["uniform", "distance"], 'n_neighbors': [3, 4, 5]}]

knn_clf = KNeighborsClassifier()
grid_search = GridSearchCV(knn_clf, param_grid, cv=5, verbose=3)
grid_search.fit(X_train, y_train)

grid_search.best_params_
grid_search.best_score_

from sklearn.metrics import accuracy_score
y_pred = grid_search.predict(X_test)
accuracy_score(y_test, y_pred)


In [ ]:
from scipy.ndimage.interpolation import shift

def shift_image(image, dx, dy):
    image = image.reshape((28, 28))
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    return shifted_image.reshape([-1])


In [ ]:
X_train_augmented = [image for image in X_train]
y_train_augmented = [label for label in y_train]

for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
    for image, label in zip(X_train, y_train):
        X_train_augmented.append(shift_image(image, dx, dy))
        y_train_augmented.append(label)


In [ ]:
X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

# Shuffle
shuffle_idx = np.random.permutation(len(X_train_augmented))
X_train_augmented = X_train_augmented[shuffle_idx]
y_train_augmented = y_train_augmented[shuffle_idx]

# Gunakan best parameter dari grid search
knn_clf = KNeighborsClassifier(**grid_search.best_params_)
knn_clf.fit(X_train_augmented, y_train_augmented)


In [ ]:
y_pred = knn_clf.predict(X_test)
accuracy_score(y_test, y_pred)
